In [2]:
import pandas as pd

import sys

sys.path.append("./..")
from src.utils.get_youtube_url_official import find_urls_for_songs

In [3]:
def get_uri_for_curated(number_of_songs=4):
    df = pd.read_csv("../data/top_songs_curated.csv")
    df_missing = df[df["youtube_title"].isna()]

    df_missing_n = df_missing.head(number_of_songs)

    songs = (
        df_missing_n[["Track Name", "Artist Name(s)"]]
        .rename(columns={"Track Name": "title", "Artist Name(s)": "artist"})
        .to_dict(orient="records")
    )

    results = find_urls_for_songs(songs)
    results = pd.DataFrame.from_dict(results, orient="index")
    results = results[results["youtube_title"] != "Not found"]

    merged_df = df.merge(
        results,
        how="left",
        left_on=["Track Name", "Artist Name(s)"],
        right_on=["title", "artist"],
        suffixes=("", "_drop"),
    )

    merged_df["youtube_title"] = merged_df["youtube_title"].combine_first(
        merged_df["youtube_title_drop"]
    )
    merged_df["youtube_url"] = merged_df["youtube_url"].combine_first(
        merged_df["youtube_url_drop"]
    )
    merged_df = merged_df.drop(
        columns=["title", "artist", "youtube_title_drop", "youtube_url_drop"]
    )

    merged_df = merged_df.sort_values(["Artist Name(s)", "Track Name"])
    merged_df.to_csv("../data/top_songs_curated.csv", index=False)

In [4]:
get_uri_for_curated(number_of_songs=99)

API key loaded successfully!
Searching for 'Monkey Wrench' by Foo Fighters...
Searching for 'My Hero' by Foo Fighters...
Searching for 'Rope' by Foo Fighters...
Searching for 'The Pretender' by Foo Fighters...
Searching for 'These Days' by Foo Fighters...
Searching for 'This Is a Call' by Foo Fighters...
Searching for 'Times Like These' by Foo Fighters...
Searching for 'Walk' by Foo Fighters...
Searching for 'Wheels' by Foo Fighters...
Searching for 'A Different Corner' by George Michael...
Searching for 'Amazing' by George Michael...
Searching for 'Careless Whisper' by George Michael...
Searching for 'Careless Whisper - Remastered' by George Michael...
Searching for 'Faith - Remastered' by George Michael...
Searching for 'Fastlove, Pt. 1' by George Michael...
Searching for 'Father Figure - Remastered' by George Michael...
Searching for 'Freedom! '90 - Remastered' by George Michael...
Searching for 'I Want Your Sex - Pts. 1 & 2 Remastered' by George Michael...
Searching for 'Jesus to a